In [1]:
!ls

dma2.bit  dma2.hwh  dma_demo.ipynb


In [2]:
from pynq import Overlay

In [3]:
ol = Overlay("dma2.bit")

In [4]:
ol.ip_dict

{'dma': {'fullpath': 'dma',
  'type': 'xilinx.com:ip:axi_dma:7.1',
  'bdtype': None,
  'state': None,
  'addr_range': 65536,
  'phys_addr': 2147483648,
  'mem_id': 'S_AXI_LITE',
  'memtype': 'REGISTER',
  'gpio': {},
  'interrupts': {},
  'parameters': {'C_S_AXI_LITE_ADDR_WIDTH': '10',
   'C_S_AXI_LITE_DATA_WIDTH': '32',
   'C_DLYTMR_RESOLUTION': '125',
   'C_PRMRY_IS_ACLK_ASYNC': '0',
   'C_ENABLE_MULTI_CHANNEL': '0',
   'C_NUM_MM2S_CHANNELS': '1',
   'C_NUM_S2MM_CHANNELS': '1',
   'C_INCLUDE_SG': '0',
   'C_SG_INCLUDE_STSCNTRL_STRM': '0',
   'C_SG_USE_STSAPP_LENGTH': '0',
   'C_SG_LENGTH_WIDTH': '26',
   'C_M_AXI_SG_ADDR_WIDTH': '64',
   'C_M_AXI_SG_DATA_WIDTH': '32',
   'C_M_AXIS_MM2S_CNTRL_TDATA_WIDTH': '32',
   'C_S_AXIS_S2MM_STS_TDATA_WIDTH': '32',
   'C_MICRO_DMA': '0',
   'C_INCLUDE_MM2S': '1',
   'C_INCLUDE_MM2S_SF': '1',
   'C_MM2S_BURST_SIZE': '8',
   'C_M_AXI_MM2S_ADDR_WIDTH': '64',
   'C_M_AXI_MM2S_DATA_WIDTH': '64',
   'C_M_AXIS_MM2S_TDATA_WIDTH': '32',
   'C_INCLUDE_MM2S

In [5]:
ol.dma?

In [6]:
dma = ol.dma
dma_send = ol.dma.sendchannel
dma_recv = ol.dma.recvchannel

In [7]:
from pynq import allocate
import numpy as np

In [8]:
data_size = 1000
input_buffer = allocate(shape=(data_size,), dtype=np.uint32)

In [9]:
for i in range(data_size):
    input_buffer[i] = i + 0xcafe0000

In [10]:
for i in range(10):
    print(hex(input_buffer[i]))

0xcafe0000
0xcafe0001
0xcafe0002
0xcafe0003
0xcafe0004
0xcafe0005
0xcafe0006
0xcafe0007
0xcafe0008
0xcafe0009


In [11]:
dma_send.transfer(input_buffer)

In [12]:
output_buffer = allocate(shape=(data_size,), dtype=np.uint32)

In [13]:
for i in range(10):
    print('0x' + format(output_buffer[i], '02x'))

0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00


In [14]:
dma_recv.transfer(output_buffer)

In [15]:
for i in range(10):
    print('0x' + format(output_buffer[i], '02x'))

0xcafe0000
0xcafe0001
0xcafe0002
0xcafe0003
0xcafe0004
0xcafe0005
0xcafe0006
0xcafe0007
0xcafe0008
0xcafe0009


In [16]:
print("Arrays are equal: {}".format(np.array_equal(input_buffer, output_buffer)))

Arrays are equal: True


In [17]:
dma_recv.error?

In [18]:
dma_recv.error

False

In [19]:
dma_recv.idle?

In [20]:
dma_recv.idle

True

In [21]:
dma_recv.transfer(output_buffer)

In [22]:
dma_recv.idle

False

In [23]:
dma_recv.transfer(output_buffer)

RuntimeError: DMA channel not idle

In [24]:
dma_recv.running?

In [25]:
dma_recv.running

True

In [26]:
dma.register_map

RegisterMap {
  MM2S_DMACR = Register(RS=1, Reset=0, Keyhole=0, Cyclic_BD_Enable=0, IOC_IrqEn=0, Dly_IrqEn=0, Err_IrqEn=0, IRQThreshold=1, IRQDelay=0),
  MM2S_DMASR = Register(Halted=0, Idle=1, SGIncld=0, DMAIntErr=0, DMASlvErr=0, DMADecErr=0, SGIntErr=0, SGSlvErr=0, SGDecErr=0, IOC_Irq=1, Dly_Irq=0, Err_Irq=0, IRQThresholdSts=0, IRQDelaySts=0),
  MM2S_CURDESC = Register(Current_Descriptor_Pointer=0),
  MM2S_CURDESC_MSB = Register(Current_Descriptor_Pointer=0),
  MM2S_TAILDESC = Register(Tail_Descriptor_Pointer=0),
  MM2S_TAILDESC_MSB = Register(Tail_Descriptor_Pointer=0),
  MM2S_SA = Register(Source_Address=1938620416),
  MM2S_SA_MSB = Register(Source_Address=0),
  MM2S_LENGTH = Register(Length=4000),
  SG_CTL = Register(SG_CACHE=0, SG_USER=0),
  S2MM_DMACR = Register(RS=1, Reset=0, Keyhole=0, Cyclic_BD_Enable=0, IOC_IrqEn=0, Dly_IrqEn=0, Err_IrqEn=0, IRQThreshold=1, IRQDelay=0),
  S2MM_DMASR = Register(Halted=0, Idle=0, SGIncld=0, DMAIntErr=0, DMASlvErr=0, DMADecErr=0, SGIntErr=0, SG

In [27]:
print("Input buffer address   :", hex(input_buffer.physical_address))
print("Output buffer address  :", hex(output_buffer.physical_address))
print("---")
print("DMA Source address     :", hex(dma.register_map.MM2S_SA.Source_Address))
print("DMA Destination address:", hex(dma.register_map.S2MM_DA.Destination_Address))

Input buffer address   : 0x738d0000
Output buffer address  : 0x738d1000
---
DMA Source address     : 0x738d0000
DMA Destination address: 0x738d1000


In [28]:
del input_buffer, output_buffer